<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#CatBoostclassifier" data-toc-modified-id="CatBoostclassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostclassifier</a></span></li></ul></li><li><span><a href="#BERT" data-toc-modified-id="BERT-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet,stopwords

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
count_toxic = 0
count_notoxic = 0
for i in df['toxic']:
    if i==0:
        count_notoxic +=1
    else:
        count_toxic +=1
print('Процент позитивных комментариев:','{0:.000%}'.format(count_notoxic/len(df['toxic'])))
print('Процент негативных комментариев:','{0:.000%}'.format(count_toxic/len(df['toxic'])))

Процент позитивных комментариев: 90%
Процент негативных комментариев: 10%


In [6]:
nltk.download('wordnet') 
import nltk
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
def clear(text):
    text = text.lower()
    text  = re.sub(r'[^a-zA-z]',' ',str(text))
    text = ' '.join(text.split())
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    
               

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
df['lemm_text'] = df['text'].apply(clear)

In [8]:
df

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,and for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


Визуально некоторые слова поменялись,но у меня остались сомнения в успешной лемматизации.

## Обучение

In [9]:
features = df['lemm_text']
target = df['toxic']

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
#count_vect = CountVectorizer(stop_words=stop_words)
#features = count_vect.fit_transform(features)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
features_train,features_test,target_train,target_test = train_test_split(features,target,test_size = 0.25,
                                                                         random_state=12345,stratify=target)

In [11]:
count_tf_idf = TfidfVectorizer(stop_words = stop_words)
features_train_tfidf  = count_tf_idf.fit_transform(features_train)
features_test_tfidf  = count_tf_idf.transform(features_test)

### LogisticRegression

In [12]:
model = LogisticRegression(class_weight='balanced')
model.fit(features_train_tfidf,target_train)

LogisticRegression(class_weight='balanced')

In [13]:
pred_train = model.predict(features_train_tfidf)
pred_test = model.predict(features_test_tfidf)
print('f1_score на обучающей выборке:',f1_score(target_train,pred_train))
print('f1_score на тестовой выборке:',f1_score(target_test,pred_test))

f1_score на обучающей выборке: 0.8228986631864837
f1_score на тестовой выборке: 0.7557874144114769


### RandomForestClassifier

In [ ]:
pipeline_rd = Pipeline(
 steps=[
 ("tfidf", TfidfVectorizer()),
 ("classifier", RandomForestClassifier(class_weight='balanced',random_state=12345)), # final estimator is a classifier
 ]
)



In [ ]:
%%time
random_search = {'classifier__criterion': ['entropy', 'gini'],
               'classifier__max_depth': range(1, 100, 5),
               'classifier__max_features': ['auto', 'sqrt','log2', None],
               'classifier__min_samples_leaf': [6,8,14,16,18,20],
               'classifier__min_samples_split': [3,5,7,9,14],
               'classifier__n_estimators': range(1, 100, 5),
               'classifier__bootstrap':[True]}



search = RandomizedSearchCV(pipeline_rd, param_distributions = random_search,n_jobs = -1,n_iter=50,
                               cv = 4)
search.fit(features,target)
best_params = search.best_params_
print(best_params)

In [14]:
%%time
model = RandomForestClassifier(n_estimators= 45,max_depth = 60,class_weight='balanced',random_state=12345,min_samples_split=17,min_samples_leaf= 4,
                               max_features=None, criterion = 'entropy', bootstrap= True)
model.fit(features_train_tfidf,target_train)
pred_train = model.predict(features_train_tfidf)
pred_test = model.predict(features_test_tfidf)
print('f1_score на обучающей выборке:',f1_score(target_train,pred_train))
print('f1_score на тестовой выборке:',f1_score(target_test,pred_test))

f1_score на обучающей выборке: 0.812156812667603
f1_score на тестовой выборке: 0.7104603614298907
CPU times: user 21min 49s, sys: 2.11 s, total: 21min 51s
Wall time: 21min 52s


In [ ]:
model = RandomForestClassifier(class_weight='balanced',random_state=12345,n_estimators= 460,min_samples_split=14, min_samples_leaf= 4,
                               max_features=None, max_depth=10, criterion = 'entropy', bootstrap= True)
model.fit(features_train_tfidf,target_train)
pred_train = model.predict(features_train_tfidf)
pred_test = model.predict(features_test_tfidf)
print('f1_score на обучающей выборке:',f1_score(target_train,pred_train))
print('f1_score на тестовой выборке:',f1_score(target_test,pred_test))

### CatBoostclassifier

In [ ]:
pipeline_cat = Pipeline(
 steps=[
     ('vect', CountVectorizer()),
 ("tfidf", TfidfTransformer()),
 ("classifier", CatBoostClassifier(random_state=12345)), # final estimator is a classifier
 ]
)


In [ ]:
params = {'classifier__learning_rate': [0.01,0.03,0.05,0.3,0.5],
        'classifier__depth': range(1,10),
        'classifier__l2_leaf_reg': range(1,20),
        'classifier__n_estimators' : range(1,50),
        'classifier__grow_policy' : ['Depthwise','SymmetricTree']}
rd = RandomizedSearchCV(estimator=pipeline_cat, param_distributions=params, cv=5, n_jobs=-1, scoring='f1',verbose=False)
rd.fit(features_train,target_train)
parameters = rd.best_params_
print(parameters)

In [15]:
model = CatBoostClassifier(n_estimators=100, learning_rate=0.03, l2_leaf_reg=8, max_bin = 3,grow_policy='Depthwise',
                           depth=10)
model.fit(features_train_tfidf,target_train)
pred_train = model.predict(features_train_tfidf)
pred_test = model.predict(features_test_tfidf)
print('f1_score на обучающей выборке:',f1_score(target_train,pred_train))
print('f1_score на тестовой выборке:',f1_score(target_test,pred_test))

0:	learn: 0.6609093	total: 3.34s	remaining: 5m 30s
1:	learn: 0.6302246	total: 6.13s	remaining: 5m
2:	learn: 0.6025496	total: 8.88s	remaining: 4m 47s
3:	learn: 0.5766563	total: 11s	remaining: 4m 24s
4:	learn: 0.5523812	total: 13.8s	remaining: 4m 21s
5:	learn: 0.5294797	total: 16s	remaining: 4m 10s
6:	learn: 0.5085418	total: 18.8s	remaining: 4m 9s
7:	learn: 0.4897451	total: 21.6s	remaining: 4m 8s
8:	learn: 0.4716246	total: 23.8s	remaining: 4m
9:	learn: 0.4552234	total: 26.4s	remaining: 3m 57s
10:	learn: 0.4388797	total: 29.2s	remaining: 3m 56s
11:	learn: 0.4244294	total: 32.1s	remaining: 3m 55s
12:	learn: 0.4095933	total: 35.1s	remaining: 3m 54s
13:	learn: 0.3962034	total: 37.9s	remaining: 3m 52s
14:	learn: 0.3840544	total: 40.5s	remaining: 3m 49s
15:	learn: 0.3730895	total: 44s	remaining: 3m 51s
16:	learn: 0.3632407	total: 47.2s	remaining: 3m 50s
17:	learn: 0.3534635	total: 49.8s	remaining: 3m 46s
18:	learn: 0.3448342	total: 52s	remaining: 3m 41s
19:	learn: 0.3360397	total: 54.3s	remain

## Выводы

Для классификации текста на позитивные и негативные комментарии, была проведена лемматизация WordNetLemmatizer()+POS.  
Разделение на тестовую и тренировочную с учетом дисбаланса класса.  
Наилучший результат дала логистическая регрессия на полном датасете  
f1_score на обучающей выборке: 0.8228986631864837  
f1_score на тестовой выборке: 0.7557874144114769  
Результаты RandomForestClassifier:  
f1_score на обучающей выборке: 0.812156812667603  
f1_score на тестовой выборке: 0.7104603614298907  
Результаты CatBoostClassifier:  
f1_score на обучающей выборке: 0.6005500364820114  
f1_score на тестовой выборке: 0.5987882867721306

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны